# Building and Testing Recommender Systems With Surprise

Surprise is a Python scikit building and analyzing recommender systems that deal with explicit rating data.

<http://surpriselib.com/>

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

In [2]:
movies = pd.read_csv(r'Movie_data.csv')

movies.head(100)

,Unnamed: 0,user_id,item_id,rating,timestamp,title
0,0,0,50,5,881250949,Star Wars (1977)
1,1,290,50,5,880473582,Star Wars (1977)
2,2,79,50,4,891271545,Star Wars (1977)
3,3,2,50,5,888552084,Star Wars (1977)
4,4,8,50,5,879362124,Star Wars (1977)
...,...,...,...,...,...,...
95,95,23,50,4,874784440,Star Wars (1977)
96,96,433,50,5,880585885,Star Wars (1977)
97,97,125,50,5,892836362,Star Wars (1977)
98,98,320,50,4,884749227,Star Wars (1977)


In [3]:
users = movies.groupby('user_id')['rating'].count().reset_index().sort_values('rating', ascending=False)
users

,user_id,rating
405,405,737
655,655,685
13,13,636
450,450,540
276,276,518
...,...,...
571,571,20
19,19,20
888,888,20
895,895,20


Most of the users gave less than 5 ratings, and very few users gave many ratings, although the most productive user have given 13,602 ratings.

I'm sure you have noticed that the above two charts share the same distribution. The number of ratings per movie and the bnumber of ratings per user decay exponentially.

To reduce the dimensionality of the dataset, we will filter out rarely rated movies and rarely rating users.

In [4]:
min_movie_ratings = 50
filter_movies = movies['title'].value_counts() > min_movie_ratings
filter_movies = filter_movies[filter_movies].index.tolist()

min_user_ratings = 50
filter_users = movies['user_id'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df = movies[(movies['title'].isin(filter_movies)) & (movies['user_id'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(movies.shape))
print('The new data frame shape:\t{}'.format(df.shape))

The original data frame shape:	(100003, 6)
The new data frame shape:	(73134, 6)


In [5]:
df['title'].nunique()

596

In [6]:
df

,Unnamed: 0,user_id,item_id,rating,timestamp,title
1,1,290,50,5,880473582,Star Wars (1977)
2,2,79,50,4,891271545,Star Wars (1977)
3,3,2,50,5,888552084,Star Wars (1977)
4,4,8,50,5,879362124,Star Wars (1977)
5,5,274,50,5,878944679,Star Wars (1977)
...,...,...,...,...,...,...
97645,97645,493,881,1,884130009,Money Talks (1997)
97647,97647,697,881,2,882621523,Money Talks (1997)
97649,97649,629,881,3,880116023,Money Talks (1997)
97652,97652,889,881,3,880176717,Money Talks (1997)


## Surprise

To load a data set from the above pandas data frame, we will use the load_from_df() method, we will also need a Reader object, and the rating_scale parameter must be specified. The data frame must have three columns, corresponding to the user ids, the item ids, and the ratings in this order. Each row thus corresponds to a given rating.

In [7]:
from surprise import Reader
from surprise import Dataset

from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

In [8]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'title', 'rating']], reader)

The Reader class is used to parse a file containing ratings.

Such a file is assumed to specify only one rating per line, and each line needs to respect the following structure:

In [11]:
benchmark = []

# Iterate over all algorithms
for algorithm in [SVD(),SlopeOne(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:   # NMF()
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    print(algorithm)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
       

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


In [ ]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results

## Train and Predict

SVDpp produces the best results, however it it very time consuming. lets train and predict using KNNBaseline

In [9]:
print('Using KNNBaseline')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = KNNBaseline()
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using KNNBaseline
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.90848826, 0.90715171, 0.90899853]),
 'fit_time': (0.26741909980773926, 0.2830533981323242, 0.27222609519958496),
 'test_time': (3.933581829071045, 3.7203969955444336, 3.658027410507202)}

In [10]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=101)
algo = KNNBaseline()

predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9022


0.9021662302921266

To inspect our predictions in details, we are going to build a pandas data frame with all the predictions. The following code were largely taken from this notebook.

In [11]:
trainset = algo.trainset
print(algo.__class__.__name__)

KNNBaseline


To inspect our predictions in details, we are going to build a pandas data frame with all the predictions.

In [12]:
# def get_Iu(user_id):
# #     """ return the number of items rated by given user args: 
# #       user_id: the id of the user 
# #       Item_User : the number of items rated by the user
# #     """
#     try:
#         return len(trainset.ur[trainset.to_inner_uid(user_id)])
#     except ValueError: # user was not part of the trainset
#         return 0
    
# def get_Ui(item_id):
# #     """ return number of users that have rated given item :
# #       item_id: the title of the movie
# #       User_Item: the number of users that have rated the item.
# #     """
#     try: 
#         return len(trainset.ir[trainset.to_inner_iid(item_id)])
#     except ValueError:
#         return 0
    
# df1 = pd.DataFrame(predictions, columns=['user_id', 'item_id', 'real', 'est', 'details'])
# df1['Item_USer'] = df1.user_id.apply(get_Iu)
# df1['User_Item'] = df1.item_id.apply(get_Ui)
# df1['err'] = abs(df1.est - df1.real)

In [13]:
# best_predictions = df1.sort_values(by='err')[:20]
# worst_predictions = df1.sort_values(by='err')[-20:]

In [69]:
# # https://surprise.readthedocs.io/en/stable/FAQ.html
# from collections import defaultdict
# def get_top_n(predictions, n=10):
    
#     # First map the predictions to each user.
#     top_n = defaultdict(list)
#     for uid, iid, true_r, est, _ in predictions:
#         top_n[uid].append((iid, est))

#     # Then sort the predictions for each user and retrieve the k highest ones.
#     for uid, user_ratings in top_n.items():
#         user_ratings.sort(key=lambda x: x[1], reverse=True)
#         top_n[uid] = user_ratings[:n]

#     return top_n

# top_n = get_top_n(predictions, n=5)
# rec = pd.DataFrame(top_n.items(),columns=['User_id','Top 5 movies and ratings'])
# list1 = list(rec[rec['User_id']==290]['Top 5 movies and ratings'])
# pd.DataFrame(list1[0], columns=['Movie name','Rating'])

In [138]:
# Input the user_id for which you need recommendations
userId = input("Enter the user id to get recommendations : ")

# Input N, while N = number of recommendations you want
n = input("Enter N, while N is the number of recommendations you want : ")

Enter the user id to get recommendations : 100
Enter N, while N is the number of recommendations you want : 20


In [140]:
# get a list of all movie ids
iids = df['item_id'].unique()

# get a list of iids that user_id has rated
iids_userId = df.loc[df['user_id'] == userId, 'item_id']

# Remove the iids that user_id has rated from the list of all movie data 
iids_to_pred = np.setdiff1d(iids,iids_userId)


testset = [[userId,item_id,5.] for item_id in iids_to_pred]
predictions = algo.test(testset)

In [141]:
pred_ratings = np.array([pred.est for pred in predictions])

# For top 'n' recommendations
i_max = np.argpartition(pred_ratings, -int(n))[-int(n):]

# For top recommendation
# i_max = pred_ratings.argmax()

iid = iids_to_pred[i_max]


# item = list(df[df['item_id']==iid]['title'])[0]
# item
# print('Top movie suggested for user_id : {} is {}'.format(100, item))

In [143]:
g = pd.DataFrame()
for i in range(0, int(n)):
    l = df[df['item_id']==iid[i]]['title'].head(1)
    x = pd.DataFrame(l)
    g = g.append(x)

In [144]:
g.head()

,title
1125,Kolya (1996)
20541,Mimic (1997)
57210,Beavis and Butt-head Do America (1996)
72659,Breaking the Waves (1996)
47986,Raising Arizona (1987)


In [104]:
x.argpartition(2)

array([0, 4, 1, 3, 2, 5, 6, 7, 8, 9])

In [122]:
x = np.array([0, 4, 1, 3, 2, 5, 6, 7, 8, 9])

In [125]:
ind = np.argpartition(x, -6)[-6:]

In [126]:
ind

array([1, 5, 6, 7, 8, 9])